In [13]:
import torch
import itertools
from torch.utils.data import DataLoader

from dataset import FontDataset
from models import Discriminator, ContentEncoder, StyleEncoder, Decoder
from losses import DisLoss
from accelerate import Accelerator

In [14]:
lr = 0.0002
beta1 = 0.5

In [15]:
accelerator = Accelerator()
dataset = FontDataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
discriminator = Discriminator()
content_encoder = ContentEncoder(G_ch=64)
style_encoder = StyleEncoder(G_ch=64)
decoder = Decoder(G_ch=64, nEmbedding=1024)

TypeError: FontDataset.__init__() missing 1 required positional argument: 'path'

In [12]:
optimizer_G = torch.optim.Adam(itertools.chain(
    content_encoder.parameters(),
    style_encoder.parameters(),
    decoder.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

criterionD = DisLoss()

In [ ]:
dataloader, content_encoder, style_encoder, decoder, discriminator, optimizer_D, optimizer_G = accelerator.prepare(
    dataloader, content_encoder, style_encoder, decoder, discriminator, optimizer_D, optimizer_G)

In [ ]:
for data in dataloader:
    optimizer_D.zero_grad()
    optimizer_G.zero_grad()

    cont, residulte_features = content_encoder(data['content'])
    style_emb, style_fc, residual_features_style = style_encoder(data['style'])
    img_print2write = decoder(cont, residulte_features, style_emb, style_fc, residual_features_style)

    real_out = discriminator(data['target'])
    fake_out = discriminator(img_print2write)

    loss_D = criterionD(real_out, True) + criterionD(fake_out, False)

